In [1]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random
import matplotlib.pyplot as plt
from gym import spaces

In [2]:
# Neural network for NAF
class NAFNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(NAFNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc_value = nn.Linear(256, 1)
        self.fc_mu = nn.Linear(256, action_size)
        self.fc_l = nn.Linear(256, action_size * action_size)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        value = self.fc_value(x)
        mu = torch.tanh(self.fc_mu(x))*torch.tensor([1, 0.167])
        l = self.fc_l(x)

        l_matrix = l.view(-1, action_size, action_size)
        l_matrix = torch.tril(l_matrix, -1) + torch.diag_embed(torch.exp(torch.diagonal(l_matrix, dim1=-2, dim2=-1)))
        p_matrix = torch.bmm(l_matrix, l_matrix.transpose(2, 1))

        return value, mu, p_matrix

    def q_value(self, state, action):
        value, mu, p_matrix = self.forward(state)
        action_diff = action - mu
        advantage = -0.5 * torch.bmm(action_diff.unsqueeze(1), torch.bmm(p_matrix, action_diff.unsqueeze(2))).squeeze(2)
        q_value = value + advantage
        return q_value

In [3]:
# Replay buffer
class ReplayBuffer:
    def __init__(self, size):
        self.memory = deque(maxlen=size)
    
    def add(self, experience):
        self.memory.append(experience)
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

In [22]:
def train_model():
    if len(memory) < batch_size*50:
        return 

    minibatch = memory.sample(batch_size)
    states = torch.FloatTensor([e[0] for e in minibatch])
    actions = torch.FloatTensor([e[1] for e in minibatch])
    rewards = torch.FloatTensor([e[2] for e in minibatch])
    next_states = torch.FloatTensor([e[3] for e in minibatch])
    dones = torch.FloatTensor([e[4] for e in minibatch])

    q_values = naf_network.q_value(states, actions)
    next_actions = target_naf_network(next_states)[1]
    next_q_values = target_naf_network.q_value(next_states, next_actions)
    target_q_values = rewards.unsqueeze(1) + (1 - dones).unsqueeze(1) * discount_factor * next_q_values

    loss = loss_fn(q_values, target_q_values)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    for target_param, param in zip(target_naf_network.parameters(), naf_network.parameters()):
        target_param.data.copy_(tau * param.data + (1.0 - tau) * target_param.data)

In [5]:
def get_action(state, epsilon):
    if np.random.rand() < epsilon:
        return env.action_space.sample()
    state = torch.FloatTensor(state).unsqueeze(0)
    _, mu, _ = naf_network(state)
    return mu.detach().numpy()[0]

In [31]:
class cstr_env(gym.Env):

    def __init__(self):  
        self.action_space = spaces.Box(low = np.array([-1.0, -0.0167], dtype=np.float32), 
                                       high = np.array([1.0 , 0.0167], dtype=np.float32), 
                                       dtype=np.float32, shape=(2, ))   
        self.observation_space = spaces.Box(low=np.array([-1.0, -1.0], dtype=np.float32), 
                                            high=np.array([1.0, 1.0], dtype=np.float32), 
                                            dtype=np.float32, shape=(2, ))
        self.n_episode = 0 # current episode number.

        
    def is_done(self, x_next):
        done = False
        c1 = (abs(x_next[0] - self.setpoint_states[0]) < 0.01)
        c2 = (abs(x_next[1] - self.setpoint_states[1]) < 0.01)
        steady_state = c1 and c2  

        # Record the steady state status for the current step
        self.goal_state_done[self.ep_step] = steady_state
        
        if self.ep_step > 3: 
            p3 = self.goal_state_done[self.ep_step-2] 
            p2 = self.goal_state_done[self.ep_step-1] 
            p1 = self.goal_state_done[self.ep_step-0] 
            # If the last three steps were steady states, set 'done' to True
            if  p3 and p2 and p1:
                done = True  
        return done 

    
    def get_dx(self, x, u):
        CT0=300; CV=1; CF=5; CE=5*(10**4); Ck0=8.46*(10**6); CdetaH=-1.15*(10**4); CCp=0.231; CrolL=1000; CR=8.314; CA0s=4; Qs=0;
        CAs= 3.9364; Ts=303.1661;  
        Cp1=(CF/CV); Cp2=Ck0; Cp3=Cp1*(CA0s-CAs); Cp4=(-CdetaH/(CrolL*CCp)); Cp5=Cp1*(CT0-Ts); Cp6= (1/(CrolL*CCp*CV));                   
        x1, x2 = x[0], x[1]
        
        f1 = -Cp1*x1 - Cp2*np.exp(-CE/(CR*(x2+Ts)))*((x1+CAs)**2) + Cp3
        f2 = -Cp1*x2 + Cp4*Cp2*np.exp(-CE/(CR*(x2+Ts)))*((x1+CAs)**2) + Cp5 + Qs*Cp6
        g1 = Cp1
        g2 = Cp6
        dx = [f1, f2] + u*[g1, g2]
        return dx

    
    def step(self, action):
        dt = 5e-3
        self.current_u = action
        state = self.current_s
        x_next = self.current_s + dt*self.get_dx(self.current_s, action)
        done = self.is_done(x_next) 
        #reward = - np.sum((x_next - self.setpoint_states)**2) - np.sum((self.current_u - self.setpoint_actions)**2)  
        reward = -x_next[0]**2*0.01 - -x_next[1]**2*0.001 - np.sum((self.current_u - self.setpoint_actions)**2)
        
        self.previous_u = self.current_u 
        self.current_s = x_next 
        self.ep_step += 1  

        # this is the trancated condition. 
        truncated = False 
        if self.ep_step == episode_length:
            truncated = True

        if self.ep_step == episode_length-1 or done:      
            self.n_episode += 1 
        
        # if done is true i.e. terminated is equal to done. 
        terminated = done

        return x_next, reward, done


    def reset(self):
        self.ep_step = 0 
        self.current_u= None 
        self.previous_u = None 
        self.current_s = None 

        ## list of true false which stores the weather the state is near to the goal state or not. 
        self.goal_state_done = [False] * (episode_length+5)

        self.setpoint_states  =  np.array([.0, .0], dtype=float)     
        self.setpoint_actions =  np.array([.0, .0], dtype=float)

        # this is the fixed initial state. 
        state, action = np.array([0.2, -5]),  np.array([1.5, 0.1]) 
        self.current_u = action 
        self.previous_u = action  
        self.current_s = state  

        return state

In [ ]:
act_range_min = np.array([-0.3, - 10.], dtype=float)   
act_range_max = np.array([0.3, 10.], dtype=float)  

transform_range_min = np.array([-1., -1.], dtype=float)  
transform_range_max = np.array([1., 1.], dtype=float)   


def normalize_minmax_states(X:np.ndarray):
    global act_range_min, act_range_max, transform_range_min, transform_range_max
    X_std = (X-act_range_min) / (act_range_max - act_range_min)
    X_scaled = X_std * (transform_range_max - transform_range_min) +  transform_range_min 
    return X_scaled

def normalize_minmax_actions(X:np.ndarray):
    global action_min_r, action_max_r, transform_action_min_r, transform_action_max_r
    X_std = (X-action_min_r) / (action_max_r - action_min_r) 
    X_scaled = X_std * (transform_action_max_r - transform_action_min_r) +  transform_action_min_r 
    return X_scaled 

In [32]:
# Hyperparameters
learning_rate = 1e-4
discount_factor = 0.99
batch_size = 256
tau = 0.001
epsilon_decay = 0.995
epsilon_min = 0.01
memory_size = 1000000
num_episodes = 3000
episode_length = num_episodes

In [33]:
memory = ReplayBuffer(memory_size)
epsilon = 1.0
# Set up the environment
env = cstr_env()
state_size = env.observation_space.shape[0]
action_size = env.action_space.shape[0]

# Initialize networks and optimizer
naf_network = NAFNetwork(state_size, action_size)
target_naf_network = NAFNetwork(state_size, action_size)
target_naf_network.load_state_dict(naf_network.state_dict())
target_naf_network.eval()

optimizer = optim.Adam(naf_network.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

for e in range(num_episodes):
    state = env.reset()
    total_reward = 0
    done = False
    step = 0
    while not done:
        step += 1
        action = get_action(state, epsilon)
        next_state, reward, done = env.step(action)
        total_reward += reward

        memory.add((state, action, reward, next_state, done))
        state = next_state
        train_model()
        if step == 500:
            break
            
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    print(f"Episode: {e+1}/{num_episodes}, Reward: {total_reward}, done: {done}")
    print(f"state: {state}, action: {action}")

env.close()

Episode: 1/3000, Reward: -173.34823169762274, done: False
state: [0.10295698 0.17738717], action: [-0.99745595 -0.01032036]
Episode: 2/3000, Reward: -153.38913096057868, done: True
state: [-0.00926547 -0.00846354], action: [-0.16661574  0.0071573 ]
Episode: 3/3000, Reward: -163.47541024295748, done: False
state: [-0.01528215 -0.02200347], action: [-0.47516263 -0.01187953]
Episode: 4/3000, Reward: -180.01399643940357, done: False
state: [-0.05776386 -0.11815424], action: [ 0.44458768 -0.00850535]
Episode: 5/3000, Reward: -155.31387731072118, done: False
state: [0.07704612 0.10504004], action: [ 0.36231124 -0.01463691]
Episode: 6/3000, Reward: -154.33052858694276, done: False
state: [ 0.06753836 -0.15498023], action: [0.96333987 0.00814522]
Episode: 7/3000, Reward: -153.48339443521218, done: False
state: [ 0.02416808 -0.03720228], action: [-0.3412669   0.01045731]
Episode: 8/3000, Reward: -128.23924689517438, done: True
state: [-0.00538498  0.00028261], action: [-0.33015558  0.00922256]
